In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager as fm
import matplotlib
import pickle
import timeit
import h5py
%matplotlib inline

In [2]:
df = pd.read_csv('20121107.csv', low_memory= False)
df.columns = pd.Series(['Timestamp', 'LineID', 'Direction', 'JourneyPatternID', 'TimeFrame', 'VehicleJourneyID', 'Operator', 'Congestion', 'LON', 'LAT', 'Delay', 'BlockID', 'VehicleID', 'StopID', 'AtStop'])
df['Date'] = df['Timestamp']

In [3]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='us')

In [4]:
df['LineID'] = df['LineID'].astype('object')
df["LineID"].fillna("null", inplace=True)
df["JourneyPatternID"].fillna("null", inplace=True)

In [5]:
def f(x):
    if x[0] != 'null':
        return str(x[0][:4]).lstrip('0')
    else:
        return x[1]

df1 = pd.concat([df['JourneyPatternID'], df['LineID']], axis = 1)
df1['Route_number'] = df1.apply(f, axis=1)

In [6]:
df = pd.concat([df,df1['Route_number']], axis=1)
df['Timestamp'] = df['Timestamp'].map(lambda x: str(x))
df= df.sort_values('Timestamp')
df

,Timestamp,LineID,Direction,JourneyPatternID,TimeFrame,VehicleJourneyID,Operator,Congestion,LON,LAT,Delay,BlockID,VehicleID,StopID,AtStop,Date,Route_number
0,2012-11-07 00:00:00,15,0,00151001,2012-11-06,5929,HN,0,-6.267883,53.417400,0,15105,33502,6317,1,1352246400000000,15
24,2012-11-07 00:00:00,46,0,046A0001,2012-11-06,7345,D2,0,-6.230733,53.317318,0,46011,36009,2039,1,1352246400000000,46A
25,2012-11-07 00:00:00,46,0,046A1002,2012-11-06,7405,D1,0,-6.230567,53.328918,0,46017,33398,320,1,1352246400000000,46A
26,2012-11-07 00:00:00,122,0,null,2012-11-06,1561,PO,0,-6.287245,53.360355,387,12208,33309,4322,0,1352246400000000,122
27,2012-11-07 00:00:00,49,0,00490001,2012-11-06,4188,RD,0,-6.362100,53.286167,-121,49001,38003,7181,1,1352246400000000,49
28,2012-11-07 00:00:00,171,0,017A0005,2012-11-06,6160,HN,0,-6.277417,53.416851,0,171008,33143,1012,1,1352246400000000,17A
29,2012-11-07 00:00:00,32,0,032A1001,2012-11-06,2702,CF,0,-6.243417,53.357166,-481,32007,38018,296,1,1352246400000000,32A
30,2012-11-07 00:00:00,451,0,045A1001,2012-11-06,7050,D2,0,-6.139283,53.293667,-1,451002,33086,4983,1,1352246400000000,45A
31,2012-11-07 00:00:00,7,0,null,2012-11-06,6531,D1,0,-6.221616,53.312561,0,7011,43004,4982,0,1352246400000000,7
32,2012-11-07 00:00:00,83,0,083A1001,2012-11-06,7219,HN,0,-6.278017,53.416485,0,83001,40014,332,1,1352246400000000,83A


In [7]:
'''df2=df[df.Route_number == '46A']
df2'''

"df2=df[df.Route_number == '46A']\ndf2"

In [8]:
'''df['time']=(df2['Date']/1000000-(df2['Date']/1000000).shift()).fillna(0)'''

"df['time']=(df2['Date']/1000000-(df2['Date']/1000000).shift()).fillna(0)"

In [9]:
'''df3=df2.loc[:,['AtStop','StopID','time']]'''

"df3=df2.loc[:,['AtStop','StopID','time']]"

In [10]:
'''df4=df2.loc[:,['LAT','LON']]
df4.head()'''

"df4=df2.loc[:,['LAT','LON']]\ndf4.head()"

In [11]:
'''def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c*1000
    return m
'''

'def haversine_np(lon1, lat1, lon2, lat2):\n    """\n    Calculate the great circle distance between two points\n    on the earth (specified in decimal degrees)\n\n    All args must be of equal length.    \n\n    """\n    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])\n\n    dlon = lon2 - lon1\n    dlat = lat2 - lat1\n\n    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2\n\n    c = 2 * np.arcsin(np.sqrt(a))\n    m = 6367 * c*1000\n    return m\n'

In [12]:
'''df['Distance'] = \
    (haversine_np(df.LON.shift(), df.LAT.shift(),
                 df.loc[1:, 'LON'], df.loc[1:, 'LAT'])).fillna(0)'''

"df['Distance'] =     (haversine_np(df.LON.shift(), df.LAT.shift(),\n                 df.loc[1:, 'LON'], df.loc[1:, 'LAT'])).fillna(0)"

In [13]:
#df5=df[['Distance','time']]
#df5.head()

In [14]:
#df6['speed'] = df3['Distance']/df3['time']
#df6

## json file

In [15]:
C = df[df.Route_number == '46A']
C.groupby(['VehicleJourneyID']).count()

,Timestamp,LineID,Direction,JourneyPatternID,TimeFrame,Operator,Congestion,LON,LAT,Delay,BlockID,VehicleID,StopID,AtStop,Date,Route_number
VehicleJourneyID,,,,,,,,,,,,,,,,
6745,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66
6755,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
6761,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51
6773,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51
6779,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6794,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6796,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6800,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6805,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61


In [17]:
dfc = df[df.VehicleJourneyID == 6745].sort_values('Timestamp')
dfc.to_json('20121107.json',orient='records')